In [1]:
import numpy as np
from math import gcd
import math

In [2]:
class LCG:
    def __init__(self, mod = 2**32, mult = 214013, inc = 2531011, seed = 42):
        self.modulo = mod
        self.multiplier = mult
        self.increment = inc
        self.val = seed

    def next(self):
        self.val =  (self.val * self.multiplier + self.increment) % self.modulo
        return self.val

In [3]:
def field_matrix_multiplication(A, B, mod):
    l = A.shape[0]
    m = A.shape[1]
    n = B.shape[1]
    C = np.zeros((l, n), dtype = np.int64)
    for i in range(0, l):
        for j in range(0, n):
            for k in range(0, m):
                C[i][j] = (C[i][j] + (A[i][k] * B[k][j] % mod)) % mod
    return C

In [4]:
def field_matrix_pow(A, power, mod):
    if (power == 0):
        return np.eye(A.shape[0], A.shape[1], dtype = np.int64)
    if (power == 1):
        return A % mod
    if (power % 2 == 0):
        return field_matrix_pow(field_matrix_multiplication(A, A, mod), power / 2, mod)
    return field_matrix_multiplication(A, field_matrix_pow(A, power - 1, mod), mod)

In [11]:
A = np.array([[1, 2], [4, 5]])
field_matrix_pow(A, 3, 100)
C = field_matrix_pow(A, 3, 100)
B = np.array([1, 2])
D = np.array([[1, 2], [4, 5]])
##field_matrix_multiplication(A, B.reshape(B.shape[0], 1), 100)
fl = False
for i in range(2, 2000):
    D = D.dot(A) % 1000
    
print(D == field_matrix_pow(A, 1999, 1000))

[[ True  True]
 [ True  True]]


In [6]:
class FibonacciAdditive:
    def __init__(self, seed, M, j):
        self.X = np.array(seed, dtype = np.int64) % M
        self.M = M
        self.j = j
        self.k = len(seed)
        self.A = np.zeros((len(self.X), len(self.X)), dtype = np.int64)
        self.A[len(self.X)-1][0] = 1
        for i in range(0, len(self.X) - 1):
            self.A[i][i+1] = 1
        self.A[- 1][self.k - j] = 1
        
    #def rand(self):
     #   ans = (self.X[0] + self.X[self.k - self.j])
      #  self.X = np.array([ans if (i == len(self.X) - 1) else self.X[i+1] for i in range(0, len(self.X))])
       # return ans
    def rand(self):
        #self.X = np.dot(self.A, self.X)
        newShape = (self.X.shape[0], 1)
        self.X = field_matrix_multiplication(self.A, self.X.reshape(newShape), self.M)
        return self.X[-1][0]
    def JumpAhead(self, step):
        B = field_matrix_pow(self.A, step, self.M)
        newShape = (self.X.shape[0], 1)
        self.X = field_matrix_multiplication(B, self.X.reshape(newShape), self.M)
        return self.X[-1][0]

In [7]:
seed = [1, 2, 3, 4, 5]
gen = FibonacciAdditive(seed, 32, 2)
otherGen = FibonacciAdditive(seed, 32, 2)
print(gen.rand())
print(gen.rand())
print(gen.rand())

5
7
8


In [8]:
LGen = LCG(seed = 15)
seed = np.array([LGen.next() for _ in range(0, 10)])
gen1 = FibonacciAdditive(seed, 2**32, 5)
gen2 = FibonacciAdditive(seed, 2**32, 5)
for _ in range(1000):
    gen1.rand()
print(gen1.rand())
print(gen2.JumpAhead(1001))

3544195055
3544195055


/home/andrei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in long_scalars
  if __name__ == '__main__':


In [9]:
gen3 = FibonacciAdditive(seed, 2**32, 5)
gen3.JumpAhead(10**9)

/home/andrei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in long_scalars
  if __name__ == '__main__':


2772760916

In [10]:
def phi(n):
    count = 0
    for k in range(1, n + 1):
        if (gcd(n, k) == 1):
            count += 1
    return count

In [11]:
def pow_mod(x, power, mod):
    x = x % mod
    if (power == 0):
        return 1
    if (power % 2 == 0):
        return pow_mod(x**2 % mod, power / 2, mod)
    return x * pow_mod(x, power - 1, mod) % mod

In [26]:
def pow_h(x, y, p):
    ans = 1
    x = x % p
    if (x == 0):
        return 0
    
    while (y > 0):
        if ((y & 1) == 1):
            ans = (ans * x) % p
        y = y >> 1
        x = (x * x) % p 
    
    return ans

In [67]:
phi(524287)

524286

In [68]:
class FibonacciMult:
    def __init__(self, seed, M, j):
        self.X = np.array(seed, dtype = int) % M 
        self.M = M
        self.j = j
        self.k = len(seed)
        self.powers = np.eye(self.k, dtype = int)
        self.phi = M - 1
        self.A = np.zeros((len(self.X), len(self.X)), dtype = int)
        self.A[len(self.X)-1][0] = 1
        for i in range(0, len(self.X) - 1):
            self.A[i][i+1] = 1
        self.A[- 1][self.k - j] = 1
        
    def stupid_rand(self):
        ans = (self.X[0] * self.X[self.k - self.j]) % self.M
        self.X = np.array([ans if (i == len(self.X) - 1) else self.X[i+1] for i in range(0, len(self.X))])
        return ans
    
    def rand(self):
        #self.powers = self.A.dot(self.powers)
        self.powers = field_matrix_multiplication(self.A, self.powers, self.phi)
        ans = 1
        for i in range(0, self.k):
            ans =  ans * pow(int(self.X[i]), int(self.powers[-1][i]), int(self.M)) % self.M
        return ans
    def JumpAhead(self, step):
        B = field_matrix_pow(self.A, step, self.phi)
        self.powers = field_matrix_multiplication(B, self.powers, self.phi)
        ans = 1
        for i in range(0, self.k):
            ans = ans * pow(int(self.X[i]), int(self.powers[-1][i]),  int(self.M)) % self.M
        return ans
    def GetPowers(self):
        return self.powers
    def GetX(self):
        return self.X
    def GetM(self):
        return self.M

In [69]:
gen3 = FibonacciMult([2, 3, 4, 5, 6], 524287, 3)
gen6 = FibonacciMult([2, 3, 4, 5, 6], 524287, 3)
for _ in range(1, 20):
    gen6.stupid_rand()
print(gen6.stupid_rand())
print(gen3.JumpAhead(20))
gen3.GetPowers()
gen6.GetM()

331489
331489


524287

In [70]:
print("seed:", seed)
gen4 = FibonacciMult(seed, 524287, 5)
gen5 = FibonacciMult(seed, 524287, 5)
step = 110
for i in range(1, step ):
    gen4.rand()
print(gen4.rand())
print(gen5.JumpAhead(step))

seed: [   5741206  334604033 3920686528  192599427 4277529146  338315285
 3607911044 3533217591 4031536414 2404876137]
473279
473279


In [71]:
gen5.JumpAhead(10**9)

76359